In [1]:
!pip install selenium
!apt-get update
!apt-get install chromium-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 26.4 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,341 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,712 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubun

In [2]:
import time
import logging
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome, ChromeOptions
import pandas as pd
from tqdm import tqdm

# Настройка логирования
log_file = "links_log.txt"
with open(log_file, "w") as f:
    f.write("Логирование начато\n")

logging.getLogger().handlers.clear()

logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    encoding="utf-8"
)

In [3]:
def web_driver():
    options = ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1200")
    options.add_argument("--disable-dev-shm-usage")
    return Chrome(options=options)

In [4]:
def parse_car_details(url, browser):
    """ Функция парсит данные одного объявления"""
    try:
        browser.get(url)
        time.sleep(2)
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        # Извлечение марки, модели и версии
        breadcrumbs = soup.find('div', {'data-marker': 'breadcrumbs'})
        if breadcrumbs:
            items = breadcrumbs.find_all('span', {'itemprop': 'name'})
            if len(items) >= 4:
                brand = items[-3].text.strip()
                model = items[-2].text.strip()
                version = items[-1].text.strip()
            else:
                brand, model, version = None, None, None
        else:
            brand, model, version = None, None, None

        # Извлечение цены
        price_span = soup.find('span', {'data-marker': 'item-view/item-price'})
        price = price_span.text.strip().replace('\xa0', ' ') if price_span else None

        # Извлечение характеристик
        params = {}
        params_block = soup.find('div', {'data-marker': 'item-view/item-params'})
        if params_block:
            for param in params_block.find_all('li'):
                key = param.find('span', {'class': 'styles-module-noAccent-l9CMS'})
                if key:
                    key_text = key.text.strip().replace(':', '')
                    value_text = param.text.replace(key.text, '').strip()
                    params[key_text] = value_text

        # Извлечение адреса
        address_span = soup.select_one('div[itemprop="address"] span.style-item-address__string-wt61A')
        address = address_span.text.strip() if address_span else None

        return {
            'URL': url,
            'Brand': brand,
            'Model': model,
            'Version': version,
            'Price': price,
            'Address': address,
            **params
        }

    except Exception as e:
        logging.error(f"Ошибка при парсинге {url}: {e}")
        return None

def get_ad_links(browser, page, base_url):
    """Получает ссылки на объявления с указанной страницы, делая до 3 попыток при необходимости (если с первого раза не получилось)"""
    max_retries = 3
    for attempt in range(1, max_retries + 1):
        try:
            listing_url = f'https://www.avito.ru/all/avtomobili?p={page}'
            browser.get(listing_url)
            time.sleep(3)
            soup = BeautifulSoup(browser.page_source, 'html.parser')

            ad_elements = soup.find_all('a', attrs={'data-marker': 'item-title'})
            ad_links = [base_url + a.get('href') for a in ad_elements if a.get('href')]

            if ad_links:
                return ad_links
            else:
                logging.warning(f"Попытка {attempt}: На странице {page} не найдено объявлений.")
                time.sleep(3)

        except Exception as e:
            logging.error(f"Ошибка при загрузке страницы {page}: {e}")

    logging.error(f"Не удалось получить ссылки на объявления с {page} страницы после {max_retries} попыток.")
    return []

In [5]:
FROM = 1 # Страница с которой начинаем (включительно)
TO = 6 # Страница которой заканчиваем (включительно)
SAVE_EVERY = 5 # Как часто сохраняем

def parse_avito():
    """Парсит объявления и сохраняет данные в CSV каждые 5 страниц."""
    browser = web_driver()
    base_url = 'https://www.avito.ru'
    car_data = []
    pages_processed = 0

    logging.info("Начало парсинга")

    for page in tqdm(range(FROM, TO+1), desc="Обработка страниц"):
        try:
            ad_links = get_ad_links(browser, page, base_url)

            if not ad_links:
                continue

            for link in tqdm(ad_links, desc=f"Объявления на стр. {page}", leave=False):
                data = parse_car_details(link, browser)
                if data:
                    car_data.append(data)

            pages_processed += 1
            logging.info(f"Страница {page} обработана, собрано {len(ad_links)} ссылок")

            # Сохранение CSV каждые несколько страниц
            if pages_processed % SAVE_EVERY == 0:
                df = pd.DataFrame(car_data)
                filename = f'cars_data_part_{page}.csv'
                df.to_csv(filename, index=False)
                logging.info(f"Сохранено {len(car_data)} записей в {filename}")
                car_data = []

        except Exception as e:
            logging.error(f"Ошибка на странице {page}: {e}")

    # Если остались несохраненные данные:
    if car_data:
        df = pd.DataFrame(car_data)
        df.to_csv('cars_data_final.csv', index=False)
        logging.info(f"Финальное сохранение {len(car_data)} записей в cars_data_final.csv")

    browser.quit()
    logging.info("Парсинг завершен!")

    logging.shutdown()

parse_avito()

Обработка страниц: 100%|██████████| 6/6 [58:42<00:00, 587.15s/it]
